<a href="https://colab.research.google.com/github/Lavanya8484/Celebal_Summer_Internship/blob/main/Week-5/HousePredictionAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas numpy matplotlib seaborn scikit-learn


In [19]:
import pandas as pd

# Load training and test datasets
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

print(train.shape)
print(test.shape)
train.head()


(1460, 81)
(1459, 80)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [20]:
print(train.info())       # Checking data types and missing values
print(train.describe())   # Summary statistics
print(train.columns)      # Listing all features

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [21]:
y = train['SalePrice']


In [22]:
train_labels = train['SalePrice']
train = train.drop(['SalePrice'], axis=1)

combined = pd.concat([train, test], keys=['train', 'test'])


In [23]:
# Handling Missing values
missing = combined.isnull().sum()
to_drop = missing[missing > 0.3 * combined.shape[0]].index.tolist()
combined.drop(columns=to_drop, inplace=True)


In [24]:
# Categorical: Fill with 'None'
cat_cols = combined.select_dtypes(include='object').columns
combined[cat_cols] = combined[cat_cols].fillna('None')

# Numerical: Fill with median
num_cols = combined.select_dtypes(exclude='object').columns
combined[num_cols] = combined[num_cols].fillna(combined[num_cols].median())


In [25]:
# Feature Engineering
# Total square footage
combined['TotalSF'] = combined['TotalBsmtSF'] + combined['1stFlrSF'] + combined['2ndFlrSF']

# Total bathrooms
combined['TotalBath'] = (combined['FullBath'] + 0.5 * combined['HalfBath'] +
                         combined['BsmtFullBath'] + 0.5 * combined['BsmtHalfBath'])

# Age of house
combined['HouseAge'] = combined['YrSold'] - combined['YearBuilt']

# Is remodeled?
combined['IsRemodeled'] = (combined['YearBuilt'] != combined['YearRemodAdd']).astype(int)

# Drop some columns
combined.drop(['GarageYrBlt', 'YearRemodAdd'], axis=1, inplace=True)


In [26]:
# Encoding categorical Variables
combined = pd.get_dummies(combined)


In [27]:
# Splitting the data back
train_processed = combined.loc['train']
test_processed = combined.loc['test']


In [28]:
# Train and Test Split Scaling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = train_processed
y = train_labels

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
test_scaled = scaler.transform(test_processed)


In [29]:
# Training the Model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Validation
val_preds = model.predict(X_val_scaled)
rmse = np.sqrt(mean_squared_error(y_val, val_preds))
print(f'Validation RMSE: {rmse:.2f}')


Validation RMSE: 28665.37


In [30]:
# Predicting
# Predict on test set
test_preds = model.predict(test_scaled)

# Create submission
submission = pd.DataFrame({
    'Id': test['Id'],
    'SalePrice': test_preds
})
submission.to_csv('submission.csv', index=False)
print("Saved submission.csv")


Saved submission.csv
